In [1]:
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision.models import vit_l_16, ViT_L_16_Weights
from torch.utils.data import DataLoader 
from torchvision.datasets import ImageFolder 
import numpy as np
from sklearn.metrics import normalized_mutual_info_score
from tqdm import tqdm
from finch import FINCH
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment

def clustering_accuracy(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1

    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    acc = w[row_ind, col_ind].sum() / y_pred.size
    return acc


def train_model(model, train_loader, epochs=10):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        train_loss = running_loss / len(train_loader)
        print(f"\nEpoch {epoch+1} | Train Loss: {train_loss:.4f}")

def extract_features(loader, model):
    features = []
    labels = []
    model.eval()
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting features"):
            images = images.to(device)
            feat = model(images)
            print("Output shape:", feat.shape)
            features.append(feat.cpu().numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)

results = []
all_image_info = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device Count: {torch.cuda.device_count()}")
print(f"Current Device: {torch.cuda.current_device()}")
print(f"Device Name: {torch.cuda.get_device_name(0)}")
print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2:.2f} MB")
print(f"Memory Reserved: {torch.cuda.memory_reserved(0)/1024**2:.2f} MB")
print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated(0)/1024**2:.2f} MB")

Using device: cuda
CUDA Available: True
Device Count: 1
Current Device: 0
Device Name: NVIDIA GeForce RTX 3060 Laptop GPU
Memory Allocated: 0.00 MB
Memory Reserved: 0.00 MB
Max Memory Allocated: 0.00 MB


In [ ]:
for trial in range(1):
    labeled_transform = transforms.Compose([
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

    labeled_folder = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\22labeled_30unlabeled\labeled"

    labeled_dataset = ImageFolder(root=labeled_folder, 
    transform=labeled_transform) 
    print(f"Total gambar dalam labeled dataset: {len(labeled_dataset)} {len(labeled_dataset.classes)}")

    batch_size = 16

    labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True) 
    print(f"Total loader gambar dalam labeled dataset: {len(labeled_loader)}")
    
    model = vit_l_16(weights=ViT_L_16_Weights.IMAGENET1K_V1)

    model.heads = torch.nn.Linear(model.heads.head.in_features, len(labeled_dataset.classes))
    model = model.to(device)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    print(model.encoder.layers[0].mlp[1])
    
    train_model(model, labeled_loader, epochs=5)

    torch.save(model.state_dict(), "vit_l_16_finetuned_22labeled_30unlabeled_trial_" + str(trial) + ".pth")

In [ ]:
for trial in range(5):

    unlabeled_transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

    unlabeled_folder = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\22labeled_30unlabeled\unlabeled"

    unlabeled_dataset = ImageFolder(root=unlabeled_folder, transform=unlabeled_transform) 
    print(f"Total gambar dalam unlabeled dataset: {len(unlabeled_dataset)}  {len(unlabeled_dataset.classes)}")

    batch_size = 16

    unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
    print(f"Total loader gambar dalam unlabeled dataset: {len(unlabeled_loader)}")
    
    model = vit_l_16(weights=None)
    model.heads = torch.nn.Identity()
    model.load_state_dict(torch.load("vit_l_16_finetuned_22labeled_30unlabeled_trial_" + str(trial) + ".pth"),strict=False)
    model = model.to(device)
    model.eval()
    
    unlabeled_features, unlabeled_true_labels = extract_features(unlabeled_loader, model)
    
    c, num_clust, req_c = FINCH(unlabeled_features, use_ann_above_samples=1000, verbose=True)
    finch_clusters = c[:,2]

    finch_nmi = normalized_mutual_info_score(unlabeled_true_labels, finch_clusters)
    finch_acc = clustering_accuracy(unlabeled_true_labels, finch_clusters)

    print(f"\nFINCH Clustering Performance:")
    print(f"Clustering Accuracy (ACC): {finch_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {finch_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(finch_clusters))}")
    
    num_clusters = len(np.unique(unlabeled_true_labels))
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_clusters = kmeans.fit_predict(unlabeled_features)

    # ----- Evaluasi -----
    kmeans_nmi = normalized_mutual_info_score(unlabeled_true_labels, kmeans_clusters)
    kmeans_acc = clustering_accuracy(unlabeled_true_labels, kmeans_clusters)

    print(f"\nK-Means Clustering Performance: {num_clusters} Clusters")
    print(f"Clustering Accuracy (ACC): {kmeans_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {kmeans_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(kmeans_clusters))}")
    
    # Simpan hasil trial ini
    results.append({
        'trial': trial,
        'FINCH_ACC': finch_acc,
        'FINCH_NMI': finch_nmi,
        'FINCH_Num_Clusters': len(np.unique(finch_clusters)),
        'KMeans_ACC': kmeans_acc,
        'KMeans_NMI': kmeans_nmi,
        'KMeans_Num_Clusters': len(np.unique(kmeans_clusters)),
    })
    
    # Simpan hasil per gambar
    for i in range(len(unlabeled_dataset)):
        path, true_label = unlabeled_dataset.samples[i]
        image_info = {
            'trial': trial,
            'image_path': path,
            'true_label': true_label,
            'finch_cluster': int(finch_clusters[i]),
            'kmeans_cluster': int(kmeans_clusters[i])
        }
        all_image_info.append(image_info)

    # Simpan file CSV per trial
    df_trial_detail = pd.DataFrame(all_image_info[-len(unlabeled_dataset):])  # ambil data dari trial ini saja
    df_trial_detail.to_csv(f'vit_l_16_finetuned_22labeled_30unlabeled_trial_{trial}_image_clustering.csv', index=False)

# Konversi ke DataFrame dan simpan ke CSV
df_results = pd.DataFrame(results)
mean_values = df_results.select_dtypes(include=np.number).mean()

mean_row = pd.DataFrame({
    'trial': ['Average'],
    'FINCH_ACC': [mean_values['FINCH_ACC']],
    'FINCH_NMI': [mean_values['FINCH_NMI']],
    'FINCH_Num_Clusters': [mean_values['FINCH_Num_Clusters']],
    'KMeans_ACC': [mean_values['KMeans_ACC']],
    'KMeans_NMI': [mean_values['KMeans_NMI']],
    'KMeans_Num_Clusters': [mean_values['KMeans_Num_Clusters']]
})

df_results = pd.concat([df_results, mean_row], ignore_index=True)

df_results.to_csv('clustering_results_vit_l_16_finetuned_22labeled_30unlabeled_FINCH_KMEANS.csv', index=False)
print("Results saved to clustering_results_vit_l_16_finetuned_22labeled_30unlabeled_FINCH_KMEANS.csv")

Total gambar dalam unlabel dataset: 3000  30
Total loader gambar dalam unlabel dataset: 188


C:\Users\HP\AppData\Local\Temp\ipykernel_21876\1539469724.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("vit_l_16_finetuned_22labeled

Output shape: torch.Size([16, 1024])


Extracting features:   1%|          | 1/188 [00:00<02:00,  1.56it/s]

Output shape: torch.Size([16, 1024])


...

Output shape: torch.Size([16, 1024])


Extracting features:  99%|█████████▉| 187/188 [01:19<00:00,  2.40it/s]

Output shape: torch.Size([8, 1024])


Extracting features: 100%|██████████| 188/188 [01:19<00:00,  2.37it/s]


Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 655 clusters
Partition 1: 131 clusters
Partition 2: 34 clusters
Partition 3: 9 clusters
Partition 4: 3 clusters

FINCH Clustering Performance:
Clustering Accuracy (ACC): 0.6987
Normalized Mutual Information (NMI): 0.7983
Number of clusters found: 34

K-Means Clustering Performance: 30 Clusters
Clustering Accuracy (ACC): 0.6300
Normalized Mutual Information (NMI): 0.7090
Number of clusters found: 30
Results saved to clustering_results_vit_l_16_pretrained_22labeled_30unlabeled_FINCH_KMEANS.csv
